In [ ]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns",50)

# time period
import datetime as dt
from datetime import timedelta
last_week = dt.datetime.today()-timedelta(days = 7)
Month_Period = last_week.strftime("%b%Y")

df1 = pd.read_excel(f"D:/ประวัติขาย BPlus/{Month_Period} ขายสด.xls")
clname_df1 = set(df1.columns)

df2 = pd.read_excel(f"D:/ประวัติขาย BPlus/{Month_Period} ขายเชื่อ.xls")
clname_df2 = set(df2.columns)

clname = list(clname_df1.intersection(clname_df2))
# append data by concatnate
df = pd.concat([df1[clname],df2[clname]],axis = 0)

# Change symbol text to integer ("$",",")
filt_df = df[["TRD_SH_CODE","TRD_SH_NAME","VAT_DATE","AR_NAME","AR_CODE","TRD_UTQNAME","TRD_U_PRC","TRD_QTY","TRD_B_AMT",
              "DI_REF"]].set_index("DI_REF")
def sub_dollar(name):
    output = pd.Series(np.where(filt_df[name].str.contains("$") == True , filt_df[name].str.replace("$",""),filt_df[name]))
    output1 = np.where(output.str.contains(",") == True , output.str.replace(",",""),output)
    return output1
filt_df["TRD_QTY"] = sub_dollar("TRD_QTY")
filt_df['TRD_B_AMT'] = sub_dollar("TRD_B_AMT")


# create round 2 decimal
filt_df["TRD_QTY"] = np.round(filt_df["TRD_QTY"].astype("float32"),2)
filt_df["TRD_B_AMT"] = np.round(filt_df["TRD_B_AMT"].astype("float32"),2)

# re- columns
filt_df = filt_df[['VAT_DATE','AR_NAME','AR_CODE','TRD_SH_CODE','TRD_SH_NAME','TRD_UTQNAME','TRD_QTY','TRD_U_PRC','TRD_B_AMT']]

# Export to excel file (include all [VAT ,NOVAT])
filt_df.to_excel(r'D:/ประวัติขาย BPlus/{monthyear} รวมขายเชื่อ กับ ขายสด.xlsx'.format(monthyear=Month_Period),
                 sheet_name='history', index = True)

# Export to excel file (Only Full VAT)
filt_df_fvat = filt_df[~filt_df.index.str.contains("N|Vย")]
print(filt_df_fvat)
filt_df_fvat.to_excel(r'D:/ประวัติขาย BPlus/{monthyear} รวมขายเชื่อ กับ ขายสด เฉพาะ VAT เต็มรูปแบบ.xlsx'.format(monthyear=Month_Period),
                 sheet_name='history', index = True)

# Export to excel file (Only abbreviate VAT)
filt_df_abvat = filt_df[filt_df.index.str.contains("Vย")]
print(filt_df_fvat)
filt_df_fvat.to_excel(r'D:/ประวัติขาย BPlus/{monthyear} รวมขายเชื่อ กับ ขายสด เฉพาะ VAT อย่างย่อ.xlsx'.format(monthyear=Month_Period),
                 sheet_name='history', index = True)